In [2]:
import pandas as pd

filepath = "/content/5대+범죄+발생현황.csv"

# ======================================================
# 1. 파일 읽기 (헤더 3줄 건너뛰기)
# ======================================================
df = pd.read_csv(filepath, skiprows=3, encoding='utf-8')


print("원본 데이터 확인")
print(df.head(10))
print(f"\n전체 행 수: {len(df)}")

# ======================================================
# 2. 컬럼명 정리
# ======================================================
df.columns = ['category', 'region', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']

# ======================================================
# 3. '소계' 행 제거 및 구 데이터만 추출
# ======================================================
df_clean = df[df['region'] != '소계'].copy()

print(f"\n '소계' 제거 후: {len(df_clean)}행")
print("\n 포함된 자치구:")
print(df_clean['region'].unique())

# ======================================================
# 4. Wide format → Long format 변환
# ======================================================
df_long = df_clean.melt(
    id_vars=['region'],
    value_vars=['2014', '2015', '2016', '2017','2018', '2019', '2020', '2021', '2022', '2023', '2024'],
    var_name='year',
    value_name='total'
)

# ======================================================
# 5. 데이터 타입 변환
# ======================================================
df_long['year'] = df_long['year'].astype(int)
df_long['total'] = pd.to_numeric(df_long['total'], errors='coerce').fillna(0).astype(int)

# ======================================================
# 6. 정렬
# ======================================================
df_long = df_long.sort_values(['region', 'year']).reset_index(drop=True)

print(f"\n 최종 데이터: {len(df_long)}행")
print(f"   연도 범위: {df_long['year'].min()} ~ {df_long['year'].max()}")
print(f"   자치구 수: {df_long['region'].nunique()}")

# ======================================================
# 7. 저장
# ======================================================
output_file = "/content/서울시_25개구_5대범죄_연도별통합.csv"
df_long.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"\n 저장 완료: {output_file}")

# ======================================================
# 8. 결과 미리보기
# ======================================================
print("\n 데이터 샘플 (각 구별 첫 해):")
print(df_long.groupby('region').first().reset_index())

print("\n 전체 데이터 처음 20행:")
print(df_long.head(20))

print("\n 연도별 서울시 전체 범죄 발생 건수:")
yearly_total = df_long.groupby('year')['total'].sum().reset_index()
yearly_total.columns = ['연도', '총 발생건수']
print(yearly_total)

원본 데이터 확인
   합계   종로구  5021  4705  4459  4057  3690  3846  3102  2712  3138  2981  2765
0  합계    중구  5231  4954  4584  4184  4030  4327  3411  2861  3071  3348  2955
1  합계   용산구  3799  3820  4137  4060  3411  3313  2969  2381  2967  3021  3322
2  합계   성동구  3582  3358  3026  2767  2457  2512  2362  2112  2194  2023  2117
3  합계   광진구  6268  5909  5322  4646  3915  4011  3601  3087  3619  3424  2870
4  합계  동대문구  4363  4720  4787  3975  3680  3692  3401  2959  3253  2957  3216
5  합계   중랑구  5353  5193  5041  4571  4288  4268  3726  3210  3599  3324  3169
6  합계   성북구  4606  4154  3744  3434  3042  2877  2567  2411  2749  2411  2184
7  합계   강북구  4030  4257  4229  3393  3437  3838  2770  2301  2832  2497  2289
8  합계   도봉구  3124  2664  2438  1999  2249  2110  2179  1860  2141  1921  1741
9  합계   노원구  5312  5130  4543  4209  4007  4153  3743  3425  3896  3567  3101

전체 행 수: 24

 '소계' 제거 후: 24행

 포함된 자치구:
['중구' '용산구' '성동구' '광진구' '동대문구' '중랑구' '성북구' '강북구' '도봉구' '노원구' '은평구' '서대문구'
 '마포구' '양천구' '강서구'